# Selected socio-economic determinants of social trust

Author: Mateusz Kasprowicz
Date: January 2024

## Load libraries

In [1]:
import random

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sklearn
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

## Constants

In [2]:
np.random.seed(0)
random.seed(0)

## Load data

In [12]:
columns_used = ["cntry", 
                "agea", 
                "eduyrs", 
                "gndr", 
                "domicil", # A big city, suburbs, town or small city, country village, etc.
                "hinctnta", # Household's total net income, all sources
                "uemp3m", # Ever unemployed and seeking work for a period more than three months
                ]

In [13]:
data = pd.read_stata(r"../data/ESS10SC_STATA/ESS10SC.dta", 
                     columns=columns_used,
                     # convert_categoricals=False,
                     )

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22074 entries, 0 to 22073
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   cntry     22074 non-null  object  
 1   agea      21028 non-null  category
 2   eduyrs    20074 non-null  category
 3   gndr      21439 non-null  category
 4   domicil   21291 non-null  category
 5   hinctnta  17232 non-null  category
 6   uemp3m    20676 non-null  category
dtypes: category(6), object(1)
memory usage: 306.8+ KB


In [15]:
data.sample(5)

,cntry,agea,eduyrs,gndr,domicil,hinctnta,uemp3m
6676,DE,77.0,NaN,Female,NaN,NaN,NaN
14824,IL,42.0,16.0,Male,Suburbs or outskirts of big city,H - 10th decile,Yes
18791,RS,61.0,12.0,Female,Town or small city,C - 3rd decile,Yes
11088,DE,51.0,13.0,Female,Town or small city,S - 6th decile,No
8420,DE,33.0,15.0,Male,Country village,R - 2nd decile,No


## Modeling

In [ ]:
# ordinal logistic regression: https://www.statsmodels.org/stable/examples/notebooks/generated/ordinal_regression.html
# https://www.statsmodels.org/dev/generated/statsmodels.miscmodels.ordinal_model.OrderedModel.html
mod_log = OrderedModel(endog=None,
                       exog=None,
                       distr="logit")
res_log = mod_log.fit(method='bfgs', disp=False)
res_log.summary()